In [8]:
from sentence_seg import SentenceSegmentor

In [9]:
SentSeg = SentenceSegmentor()

In [10]:
import pandas as pd
data = pd.read_csv('politics_news.csv')
data = data[~data['content_publish_time'].str.contains(',')]
data = data[data['content_publish_time'].str.contains('16')]

In [11]:
data_start = min(data['content_publish_time'])
data_end = max(data['content_publish_time'])
# for idx, item in data.iterrows():
#     print(item.content_publish_time)

In [12]:
from datetime import datetime
datetime.fromtimestamp(int(float(data_start)/1000))

datetime.datetime(2022, 1, 1, 0, 57, 11)

In [13]:
datetime.fromtimestamp(int(float(data_end)/1000))

datetime.datetime(2022, 8, 22, 9, 19, 45)

In [14]:
import re
title = []
uuid = []
sentence = []
sidx = []
timestamp = []
bodys = []
re_compile = '(林智堅).*?(抄襲)'
for idx, item in data.iterrows():
    body = item['body'].replace('&nbsp;', ' ')   
    if len(re.findall(re_compile, body[:30])) > 0:
        # (圖/
        sents = SentSeg.cut_sentence(body)
        sents = list(filter(lambda x: len(re.findall('報導。$',x.strip())) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('報導】。$',x.strip())) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('^&nbsp;',x.strip())) == 0, sents))
        # sents = list(filter(lambda x: len(re.findall('作者[/',x)) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('^文／',x.strip())) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('翻攝',x.strip())) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('圖片來源／',x.strip())) == 0, sents))
        sents = list(filter(lambda x: len(re.findall('\(圖/',x.strip())) == 0, sents))
        sents = [re.sub('\[.*?\]', '',  s) for s in sents]
        sents = [re.sub('\（.*?報導\）', '', s) for s in sents]
        sents = [re.sub('\【.*?報導\】', '', s) for s in sents]
        sents = [re.sub('\[.*?\]', '',  s) for s in sents]
        sents = [re.sub('\（.*?\）', '',  s) for s in sents]
        sents = [re.sub('\（中央社.*?\）', '',  s) for s in sents]
        sents = [re.sub('^更多.*?報導$', '',  s) for s in sents]
        sents = [s.strip() for s in sents if len(s.strip()) > 5]
        first_sent = ''
        for idx, s in enumerate(sents):
            first_sent += s
            if len(first_sent) > 20:
                break
        #     sentence.append(s)
        #     sidx.append(idx)
        sentence.append(first_sent)
        bodys.append(body)
        title.append(item['title'])
        uuid.append(item['uuid'])
        timestamp.append(item['content_publish_time'])

df = pd.DataFrame(uuid, columns=['uuid'])
df['title'] = title
# df['sidx'] = sidx
df['sentence'] = sentence
df['timestamp'] = timestamp
df['body'] = bodys
df.to_csv('林智堅抄襲.csv')